In [3]:
import requests
import pandas as pd
import json

# Pull Latitude and Longitude from positionstack.com based on geographical user input.

In [4]:
# User input: Country
country_input = (input("Enter a Country using a 2-letter country code (Example: US): "))

Enter a Country using a 2-letter country code (Example: US): us


In [5]:
# N/A: No need for this extra geographic variable.

# User input: State
# region_input = (input("Enter a region (City, County, State, etc.): "))

In [6]:
# User input: City

# HTTP GET Request Parameters:
# query	[Required] Specify your query as a free-text address, 
# place name or using any other common text-based location 
# identifier (e.g. postal code, city name, region name).

query_input = (input("Enter a location (City, Zip Code, etc.): "))

Enter a location (City, Zip Code, etc.): seattle


In [7]:
# Geo reference for adding to final dataframe.
geo_input = country_input  + " | " + query_input
geo_input

'us | seattle'

In [8]:
import http.client, urllib.parse

conn = http.client.HTTPConnection('api.positionstack.com')

params = urllib.parse.urlencode({
    'access_key': '1fbf7de4ce05274e9b9005659970c429',
    
    # Variables get live user input above.
    'country': country_input,
#     'region': region_input,
    'query': query_input,
        
    # "limit" of 1 ensures only 1 Lat/Lon will be passed below.    
    'limit': 1
    })

conn.request('GET', '/v1/forward?{}'.format(params))

res = conn.getresponse()
geo = res.read()
geo_str = (geo.decode('utf-8'))

print(geo.decode('utf-8'))

###########################################################
# Still needs error handling.

{"data":[{"latitude":47.673455,"longitude":-122.331537,"type":"locality","name":"Seattle","number":null,"postal_code":null,"street":null,"confidence":1,"region":"Washington","region_code":"WA","county":"King County","locality":"Seattle","administrative_area":null,"neighbourhood":null,"country":"United States","country_code":"USA","continent":"North America","label":"Seattle, WA, USA"}]}


In [9]:
# Help from AskBCS:
# Result was one long string, that only looked 
# like a dictionary, but wasn't.
# Use "type()" to identify whether or not 
# result is actually a dict.

type(geo_str)

str

In [10]:
# Convert string to json (which is a dictionary).
# geo_json
geo_json = json.loads(geo_str)

print(geo_json)

{'data': [{'latitude': 47.673455, 'longitude': -122.331537, 'type': 'locality', 'name': 'Seattle', 'number': None, 'postal_code': None, 'street': None, 'confidence': 1, 'region': 'Washington', 'region_code': 'WA', 'county': 'King County', 'locality': 'Seattle', 'administrative_area': None, 'neighbourhood': None, 'country': 'United States', 'country_code': 'USA', 'continent': 'North America', 'label': 'Seattle, WA, USA'}]}


##### Reminder: Double parentheses are functional in the for loops.

In [11]:
# # geo_json_coord = lat and lon combined into a tuple.

# geo_json_coord = geo_json['data']
# latlon_tup = []
# for x in geo_json_coord:
#     latlon_tup.append((x['latitude'], x['longitude']))

In [12]:
# Pull latitude value by itself.

geo_json_lat = geo_json['data']
var_lat = []
for x in geo_json_lat:
    var_lat.append((x['latitude']))
    
var_lat[0]

47.673455

In [13]:
# Pull longitude value by itself.

geo_json_lon = geo_json['data']
var_lon = []
for x in geo_json_lon:
    var_lon.append((x['longitude']))
    
var_lon[0]

-122.331537

# Push Latitude and Longitude to Weather API URL, based on start date and stop date from user input.

## Hourly data.

In [14]:
# List of columns, for starter dataset, hourly:

# hourly_columns = ['time',
#              'temperature_2m_degF', 
#              'relativehumidity_2m_perc', 
#              'rain_inch',
#              'snowfall_cm', 
#              'cloudcover_perc', 
#              'windspeed_10m_mph',
#              'winddirection_10m_deg']

#  We pulled data from 2000-01-01 to 2020-12-31.

Original Open Meteo API URL, hourly:

https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2000-01-01&end_date=2020-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch

In [15]:
# User input: Start Date
# start_date_input = (input("Enter the start date (YYYY-MM-DD): "))

In [16]:
# User input: End Date
# end_date_input = (input("Enter the end date (YYYY-MM-DD): "))

In [17]:
# Break URL into snippets to be assembled below, hourly:

url_snip_hr_1 = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
url_snip_hr_2 = str(var_lat[0])   # latitude, input from previous json
url_snip_hr_3 = "&longitude="
url_snip_hr_4 = str(var_lon[0])   # longitude, input from previous json



url_snip_hr_5 = "&start_date="
# url_snip_hr_6 = start_date_input   # start date, input from user
url_snip_hr_6 = "2010-01-01"   # --- OR --- static start date, for dummy data

url_snip_hr_7 = "&end_date="
# url_snip_hr_8 = end_date_input   # end date, input from user
url_snip_hr_8 = "2019-12-31"   # --- OR ---  static end date, for dummy data



url_snip_hr_9 = "&hourly="   # elements, hourly
url_snip_hr_10 = "temperature_2m,"   # data element
url_snip_hr_11 = "relativehumidity_2m,"   # data element
url_snip_hr_12 = "rain,"   # data element
url_snip_hr_13 = "snowfall,"   # data element
url_snip_hr_14 = "cloudcover,"   # data element
url_snip_hr_15 = "windspeed_10m,"   # data element
url_snip_hr_16 = "winddirection_10m"   # data element

url_snip_hr_17 = "&timezone=America%2FLos_Angeles"   # time zone
url_snip_hr_18 = "&temperature_unit=fahrenheit"   # temp unit
url_snip_hr_19 = "&windspeed_unit=mph"   # windspeed unit
url_snip_hr_20 = "&precipitation_unit=inch"   # precip unit

In [18]:
# Contatenate the url snippets, hourly.

url_hr_concat_full = \
    url_snip_hr_1 + \
    url_snip_hr_2 + \
    url_snip_hr_3 + \
    url_snip_hr_4 + \
    url_snip_hr_5 + \
    url_snip_hr_6 + \
    url_snip_hr_7 + \
    url_snip_hr_8 + \
    url_snip_hr_9 + \
    url_snip_hr_10 + \
    url_snip_hr_11 + \
    url_snip_hr_12 + \
    url_snip_hr_13 + \
    url_snip_hr_14 + \
    url_snip_hr_15 + \
    url_snip_hr_16 + \
    url_snip_hr_17 + \
    url_snip_hr_18 + \
    url_snip_hr_19 + \
    url_snip_hr_20

In [19]:
print(url_hr_concat_full)

https://archive-api.open-meteo.com/v1/era5?latitude=47.673455&longitude=-122.331537&start_date=2010-01-01&end_date=2019-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch


In [20]:
# Data comes in as one long string.
weather_hr_str = requests.get(url_hr_concat_full).text

In [21]:
# Convert string to dictionary.
weather_hr_json = json.loads(weather_hr_str)

In [22]:
# Verify that data type is dictionary.
type(weather_hr_json)

dict

In [23]:
# Convert json dictionary into dataframe.

# Standard "pd.json_normalize(weather_json)" doesn't work 
# with these nested levels. Also tried adding "record_path"
# and "meta" tags, but still didn't work.
# Found "from_records" alternative that does work.

# N/A: Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error
# Creates "pivot" table format with all entries as columns, 
# all variables as rows.
# Not what we need, but might be useful for other datasets.
# dftest3 = pd.DataFrame.from_records(weather_json['hourly']).T.reset_index()


# Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error

weather_hourly_raw = pd.DataFrame.from_records(weather_hr_json['hourly'])
weather_hourly_raw

,cloudcover,rain,relativehumidity_2m,snowfall,temperature_2m,time,winddirection_10m,windspeed_10m
0,100,0.024,93,0.0,45.6,2010-01-01T00:00,151.0,11.7
1,100,0.024,94,0.0,46.2,2010-01-01T01:00,150.0,11.3
2,100,0.016,93,0.0,46.1,2010-01-01T02:00,149.0,10.9
3,68,0.012,95,0.0,46.6,2010-01-01T03:00,153.0,12.6
4,95,0.016,94,0.0,46.5,2010-01-01T04:00,149.0,12.7
...,...,...,...,...,...,...,...,...
87643,100,0.031,95,0.0,48.2,2019-12-31T19:00,195.0,20.3
87644,100,0.059,96,0.0,49.1,2019-12-31T20:00,198.0,18.4
87645,100,0.051,96,0.0,49.7,2019-12-31T21:00,201.0,15.1
87646,100,0.043,94,0.0,50.5,2019-12-31T22:00,201.0,12.7


In [24]:
# Add geo input from user as new column, for reference.

weather_hourly_raw['geo'] = geo_input
weather_hourly_raw

,cloudcover,rain,relativehumidity_2m,snowfall,temperature_2m,time,winddirection_10m,windspeed_10m,geo
0,100,0.024,93,0.0,45.6,2010-01-01T00:00,151.0,11.7,us | seattle
1,100,0.024,94,0.0,46.2,2010-01-01T01:00,150.0,11.3,us | seattle
2,100,0.016,93,0.0,46.1,2010-01-01T02:00,149.0,10.9,us | seattle
3,68,0.012,95,0.0,46.6,2010-01-01T03:00,153.0,12.6,us | seattle
4,95,0.016,94,0.0,46.5,2010-01-01T04:00,149.0,12.7,us | seattle
...,...,...,...,...,...,...,...,...,...
87643,100,0.031,95,0.0,48.2,2019-12-31T19:00,195.0,20.3,us | seattle
87644,100,0.059,96,0.0,49.1,2019-12-31T20:00,198.0,18.4,us | seattle
87645,100,0.051,96,0.0,49.7,2019-12-31T21:00,201.0,15.1,us | seattle
87646,100,0.043,94,0.0,50.5,2019-12-31T22:00,201.0,12.7,us | seattle


## Daily data.

In [25]:
# List of columns, for starter dataset, daily:

# daily_columns = ['time',
#            'temperature_2m_degF_max',
#            'temperature_2m_degF_min',  
#            'rain_inch',
#            'snowfall_cm',
#            'precipitation_hours'
#           ]

#  We pulled data from 2000-01-01 to 2020-12-31.

Original Open Meteo API URL, daily:

https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2000-01-01&end_date=2020-12-31&daily=temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,precipitation_hours&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch

In [26]:
# Break URL into snippets to be assembled below, daily:

url_snip_dy_1 = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
url_snip_dy_2 = str(var_lat[0])   # latitude, input from previous json
url_snip_dy_3 = "&longitude="
url_snip_dy_4 = str(var_lon[0])   # longitude, input from previous json



url_snip_dy_5 = "&start_date="
# url_snip_dy_6 = start_date_input   # start date, input from user
url_snip_dy_6 = "2010-01-01"   # --- OR --- static start date, for dummy data

url_snip_dy_7 = "&end_date="
# url_snip_dy_8 = end_date_input   # end date, input from user
url_snip_dy_8 = "2019-12-31"   # --- OR ---  static end date, for dummy data



url_snip_dy_9 = "&daily="   # elements, daily
url_snip_dy_10 = "temperature_2m_max,"   # data element
url_snip_dy_11 = "temperature_2m_min,"   # data element
url_snip_dy_12 = "rain_sum,"   # data element
url_snip_dy_13 = "snowfall_sum,"   # data element
url_snip_dy_14 = "precipitation_hours"   # data element


url_snip_dy_15 = "&timezone=America%2FLos_Angeles"   # time zone
url_snip_dy_16 = "&temperature_unit=fahrenheit"   # temp unit
url_snip_dy_17 = "&windspeed_unit=mph"   # windspeed unit
url_snip_dy_18 = "&precipitation_unit=inch"   # precip unit

In [27]:
# Contatenate the url snippets, hourly.

url_dy_concat_full = \
    url_snip_dy_1 + \
    url_snip_dy_2 + \
    url_snip_dy_3 + \
    url_snip_dy_4 + \
    url_snip_dy_5 + \
    url_snip_dy_6 + \
    url_snip_dy_7 + \
    url_snip_dy_8 + \
    url_snip_dy_9 + \
    url_snip_dy_10 + \
    url_snip_dy_11 + \
    url_snip_dy_12 + \
    url_snip_dy_13 + \
    url_snip_dy_14 + \
    url_snip_dy_15 + \
    url_snip_dy_16 + \
    url_snip_dy_17 + \
    url_snip_dy_18

In [28]:
print(url_dy_concat_full)

https://archive-api.open-meteo.com/v1/era5?latitude=47.673455&longitude=-122.331537&start_date=2010-01-01&end_date=2019-12-31&daily=temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,precipitation_hours&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch


In [29]:
# Data comes in as one long string.
weather_dy_str = requests.get(url_dy_concat_full).text

In [30]:
# Convert string to dictionary.
weather_dy_json = json.loads(weather_dy_str)

In [31]:
# Verify that data type is dictionary.
type(weather_dy_json)

dict

In [32]:
# Convert json dictionary into dataframe.

# Standard "pd.json_normalize(weather_json)" doesn't work 
# with these nested levels. Also tried adding "record_path"
# and "meta" tags, but still didn't work.
# Found "from_records" alternative that does work.

# N/A: Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error
# Creates "pivot" table format with all entries as columns, 
# all variables as rows.
# Not what we need, but might be useful for other datasets.
# dftest3 = pd.DataFrame.from_records(weather_json['hourly']).T.reset_index()


# Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error

weather_daily_raw = pd.DataFrame.from_records(weather_dy_json['daily'])
weather_daily_raw

,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,time
0,24.0,0.85,0.00,49.7,45.6,2010-01-01
1,12.0,0.12,0.00,47.4,43.9,2010-01-02
2,2.0,0.02,0.00,47.4,41.6,2010-01-03
3,21.0,1.12,0.00,46.0,44.0,2010-01-04
4,15.0,0.53,0.00,47.4,45.5,2010-01-05
...,...,...,...,...,...,...
3647,5.0,0.03,0.28,43.4,38.3,2019-12-27
3648,0.0,0.00,0.00,46.0,40.5,2019-12-28
3649,0.0,0.00,0.00,48.6,43.3,2019-12-29
3650,7.0,0.05,0.00,46.9,43.4,2019-12-30


In [33]:
# Add geo input from user as new column, for reference.

weather_daily_raw['geo'] = geo_input
weather_daily_raw

,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,time,geo
0,24.0,0.85,0.00,49.7,45.6,2010-01-01,us | seattle
1,12.0,0.12,0.00,47.4,43.9,2010-01-02,us | seattle
2,2.0,0.02,0.00,47.4,41.6,2010-01-03,us | seattle
3,21.0,1.12,0.00,46.0,44.0,2010-01-04,us | seattle
4,15.0,0.53,0.00,47.4,45.5,2010-01-05,us | seattle
...,...,...,...,...,...,...,...
3647,5.0,0.03,0.28,43.4,38.3,2019-12-27,us | seattle
3648,0.0,0.00,0.00,46.0,40.5,2019-12-28,us | seattle
3649,0.0,0.00,0.00,48.6,43.3,2019-12-29,us | seattle
3650,7.0,0.05,0.00,46.9,43.4,2019-12-30,us | seattle


## aggregating hourlies into dailies for historical summary

In [34]:
weather_hourly_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87648 entries, 0 to 87647
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cloudcover           87648 non-null  int64  
 1   rain                 87648 non-null  float64
 2   relativehumidity_2m  87648 non-null  int64  
 3   snowfall             87648 non-null  float64
 4   temperature_2m       87648 non-null  float64
 5   time                 87648 non-null  object 
 6   winddirection_10m    87617 non-null  float64
 7   windspeed_10m        87648 non-null  float64
 8   geo                  87648 non-null  object 
dtypes: float64(5), int64(2), object(2)
memory usage: 6.0+ MB


In [35]:
import datetime
import numpy as np
# imports should go to the top

In [36]:
# slice of hourly to work with
hourly_temp_df = weather_hourly_raw[['time', 'cloudcover', 'relativehumidity_2m',
                                     'temperature_2m', 'windspeed_10m']].copy()

In [37]:
# convert the provided ISO string 'time' into a 'pure_date' in python datetime format
# for aggregating and joining with daily data;
# ==>> this should be done above, in intial datacleaning <<==

hourly_temp_df["pure_date"] = hourly_temp_df['time'].map(lambda x: 
                                                     datetime.datetime.fromisoformat(x[0:10]))

In [38]:
def enth(x, n):
    return x.sort_values().iloc[n]

In [61]:
step1_df = \
hourly_temp_df.groupby('pure_date').agg(humid_avg=('relativehumidity_2m', np.mean),
                                        wind_high = ('windspeed_10m', np.max),
                                        cloud_avg=('cloudcover', np.mean),
                                        temp_6= ('temperature_2m', lambda x: enth(x,18)) )
step1_df.head()

,humid_avg,wind_high,cloud_avg,temp_6
pure_date,,,,
2010-01-01,91.375000,20.5,96.625,48.7
2010-01-02,91.291667,16.6,80.250,46.7
2010-01-03,91.041667,6.8,84.375,46.0
2010-01-04,89.541667,8.0,100.000,45.3
2010-01-05,95.958333,4.7,100.000,47.1


## slicing the chunk of daily to join

In [62]:
weather_daily_raw.columns

Index(['precipitation_hours', 'rain_sum', 'snowfall_sum', 'temperature_2m_max',
       'temperature_2m_min', 'time', 'geo'],
      dtype='object')

In [63]:
# get the columns we want, rename as per specs

daily_temp_df = weather_daily_raw[['time', 'precipitation_hours', 'rain_sum',
                                   'snowfall_sum', 'temperature_2m_max']].copy()

daily_temp_df.rename(columns={'temperature_2m_max': 'temp_high'}, inplace=True)


In [64]:
# convert the provided ISO string 'time' into a 'pure_date' in python datetime format
# for aggregating and joining with daily data;
# ==>> this should be done above, in intial datacleaning <<==

daily_temp_df["pure_date"] = daily_temp_df['time'].map(lambda x: 
                                                     datetime.datetime.fromisoformat(x[0:10]))

In [65]:
daily_temp_df.drop('time', axis=1, inplace=True)

In [66]:
daily_temp_df.set_index('pure_date', inplace=True)

In [67]:
daily_temp_df.head(20)

,precipitation_hours,rain_sum,snowfall_sum,temp_high
pure_date,,,,
2010-01-01,24.0,0.85,0.0,49.7
2010-01-02,12.0,0.12,0.0,47.4
2010-01-03,2.0,0.02,0.0,47.4
2010-01-04,21.0,1.12,0.0,46.0
2010-01-05,15.0,0.53,0.0,47.4
2010-01-06,0.0,0.00,0.0,47.4
2010-01-07,0.0,0.00,0.0,44.8
2010-01-08,12.0,0.33,0.0,45.8
2010-01-09,5.0,0.06,0.0,51.6


In [68]:
step1_df = step1_df.join(daily_temp_df).head(20)

## Step 2: New columns for weather category labels.

In [69]:
step2_df = step1_df.copy()
step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high
pure_date,,,,,,,,
2010-01-01,91.375000,20.5,96.625000,48.7,24.0,0.85,0.0,49.7
2010-01-02,91.291667,16.6,80.250000,46.7,12.0,0.12,0.0,47.4
2010-01-03,91.041667,6.8,84.375000,46.0,2.0,0.02,0.0,47.4
2010-01-04,89.541667,8.0,100.000000,45.3,21.0,1.12,0.0,46.0
2010-01-05,95.958333,4.7,100.000000,47.1,15.0,0.53,0.0,47.4
2010-01-06,64.833333,7.3,91.875000,46.1,0.0,0.00,0.0,47.4
2010-01-07,43.625000,11.1,41.708333,42.3,0.0,0.00,0.0,44.8
2010-01-08,66.791667,12.3,97.958333,42.1,12.0,0.33,0.0,45.8
2010-01-09,87.791667,10.2,71.458333,49.2,5.0,0.06,0.0,51.6


In [73]:
# rain

# not_rainy = 0 hours, or  <2.5 mm total rain (Laurina changing to 0.5 inches)
# very_rainy = >=6 hour and >=10mm (Laurina changing to 2 inches)
# lightly_rainy = leftovers
step2_df['not_rainy'] = (step2_df['rain_sum'] <= 0.5) & (step2_df['precipitation_hours'] == 0)
# step2_df['not_rainy'] = step2_df['not_rainy'].astype(int)

step2_df['very_rainy'] = (step2_df['rain_sum'] >= 2) & (step2_df['precipitation_hours'] >= 6)
# step2_df['very_rainy'] = step2_df['not_rainy'].astype(int)

step2_df['lightly_rainy'] = (step2_df['not_rainy'] == 0) & (step2_df['very_rainy'] == 0)
# step2_df['lightly_rainy'] = step2_df['lightly_rainy'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,lightly_rainy
pure_date,,,,,,,,,,,
2010-01-01,91.375000,20.5,96.625000,48.7,24.0,0.85,0.0,49.7,False,False,True
2010-01-02,91.291667,16.6,80.250000,46.7,12.0,0.12,0.0,47.4,False,False,True
2010-01-03,91.041667,6.8,84.375000,46.0,2.0,0.02,0.0,47.4,False,False,True
2010-01-04,89.541667,8.0,100.000000,45.3,21.0,1.12,0.0,46.0,False,False,True
2010-01-05,95.958333,4.7,100.000000,47.1,15.0,0.53,0.0,47.4,False,False,True
2010-01-06,64.833333,7.3,91.875000,46.1,0.0,0.00,0.0,47.4,True,False,False
2010-01-07,43.625000,11.1,41.708333,42.3,0.0,0.00,0.0,44.8,True,False,False
2010-01-08,66.791667,12.3,97.958333,42.1,12.0,0.33,0.0,45.8,False,False,True
2010-01-09,87.791667,10.2,71.458333,49.2,5.0,0.06,0.0,51.6,False,False,True


In [214]:
# freezing

# freezing = >19hours with <32F: Use "temp_6" (6th highest temp of day) as cutoff.
# cold = high <60F
# hot = >6hours with >80F
# warm = leftovers

step2_df['freezing'] = (step2_df['temp_6'] <= 32)
step2_df['freezing'] = step2_df['freezing'].astype(int)

step2_df['cold'] = (step2_df['temp_high'] >= 60)
step2_df['cold'] = step2_df['cold'].astype(int)

step2_df['hot'] = (step2_df['temp_6'] >= 80)
step2_df['hot'] = step2_df['hot'].astype(int)

step2_df['warm'] = (step2_df['freezing'] == 0) & (step2_df['cold'] == 0) & (step2_df['hot'] == 0)
step2_df['warm'] = step2_df['warm'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,lightly_rainy,freezing,cold,hot,warm
pure_date,,,,,,,,,,,,,,,
2010-01-01,91.375000,20.5,96.625,100,24.0,0.85,0.0,49.7,0,0,1,0,0,1,0
2010-01-02,91.291667,16.6,80.250,100,12.0,0.12,0.0,47.4,1,1,0,0,0,1,0
2010-01-03,91.041667,6.8,84.375,100,2.0,0.02,0.0,47.4,1,1,0,0,0,1,0
2010-01-04,89.541667,8.0,100.000,100,21.0,1.12,0.0,46.0,0,0,1,0,0,1,0
2010-01-05,95.958333,4.7,100.000,100,15.0,0.53,0.0,47.4,0,0,1,0,0,1,0


In [215]:
# snow

# data used: total snow for the day
# not_snowy = 0
# very_snowy = >6"
# lightly_snowy = >0
# (unlike rain, we ignore hours)
    
step2_df['not_snowy'] = (step2_df['snowfall_sum'] == 0)
step2_df['not_snowy'] = step2_df['not_snowy'].astype(int)

step2_df['very_snowy'] = (step2_df['snowfall_sum'] >= 6)
step2_df['very_snowy'] = step2_df['very_snowy'].astype(int)

step2_df['lightly_snowy'] = (step2_df['not_snowy'] == 0) & (step2_df['very_snowy'] == 0)
step2_df['lightly_snowy'] = step2_df['lightly_snowy'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,lightly_rainy,freezing,cold,hot,warm,not_snowy,very_snowy,lightly_snowy
pure_date,,,,,,,,,,,,,,,,,,
2010-01-01,91.375000,20.5,96.625,100,24.0,0.85,0.0,49.7,0,0,1,0,0,1,0,1,0,0
2010-01-02,91.291667,16.6,80.250,100,12.0,0.12,0.0,47.4,1,1,0,0,0,1,0,1,0,0
2010-01-03,91.041667,6.8,84.375,100,2.0,0.02,0.0,47.4,1,1,0,0,0,1,0,1,0,0
2010-01-04,89.541667,8.0,100.000,100,21.0,1.12,0.0,46.0,0,0,1,0,0,1,0,1,0,0
2010-01-05,95.958333,4.7,100.000,100,15.0,0.53,0.0,47.4,0,0,1,0,0,1,0,1,0,0


In [216]:
# wind

# windy = maximum	> ?? mph
# maybe 8? mayb 12
    
step2_df['windy'] = (step2_df['wind_high'] >= 12)
step2_df['windy'] = step2_df['windy'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,lightly_rainy,freezing,cold,hot,warm,not_snowy,very_snowy,lightly_snowy,windy
pure_date,,,,,,,,,,,,,,,,,,,
2010-01-01,91.375000,20.5,96.625,100,24.0,0.85,0.0,49.7,0,0,1,0,0,1,0,1,0,0,1
2010-01-02,91.291667,16.6,80.250,100,12.0,0.12,0.0,47.4,1,1,0,0,0,1,0,1,0,0,1
2010-01-03,91.041667,6.8,84.375,100,2.0,0.02,0.0,47.4,1,1,0,0,0,1,0,1,0,0,0
2010-01-04,89.541667,8.0,100.000,100,21.0,1.12,0.0,46.0,0,0,1,0,0,1,0,1,0,0,0
2010-01-05,95.958333,4.7,100.000,100,15.0,0.53,0.0,47.4,0,0,1,0,0,1,0,1,0,0,0


In [217]:
# humidity

# average hourlies, then:
# low_humidity = <30%
# high_humidity = >60%
# medium_humidity = leftovers

step2_df['low_humidity'] = (step2_df['humid_avg'] <= 30)
step2_df['low_humidity'] = step2_df['low_humidity'].astype(int)

step2_df['high_humidity'] = (step2_df['humid_avg'] >= 60)
step2_df['high_humidity'] = step2_df['high_humidity'].astype(int)

step2_df['medium_humidity'] = (step2_df['low_humidity'] == 0) & (step2_df['high_humidity'] == 0)
step2_df['medium_humidity'] = step2_df['medium_humidity'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,...,cold,hot,warm,not_snowy,very_snowy,lightly_snowy,windy,low_humidity,high_humidity,medium_humidity
pure_date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,91.375000,20.5,96.625,100,24.0,0.85,0.0,49.7,0,0,...,0,1,0,1,0,0,1,0,1,0
2010-01-02,91.291667,16.6,80.250,100,12.0,0.12,0.0,47.4,1,1,...,0,1,0,1,0,0,1,0,1,0
2010-01-03,91.041667,6.8,84.375,100,2.0,0.02,0.0,47.4,1,1,...,0,1,0,1,0,0,0,0,1,0
2010-01-04,89.541667,8.0,100.000,100,21.0,1.12,0.0,46.0,0,0,...,0,1,0,1,0,0,0,0,1,0
2010-01-05,95.958333,4.7,100.000,100,15.0,0.53,0.0,47.4,0,0,...,0,1,0,1,0,0,0,0,1,0


In [218]:
# clouds

# average hourly "percent cloud cover"
# clear = <=33%
# cloudy = >= 67%
# partly_cloudy = leftovers

step2_df['clear'] = (step2_df['cloud_avg'] <= 30)
step2_df['clear'] = step2_df['clear'].astype(int)

step2_df['cloudy'] = (step2_df['cloud_avg'] >= 60)
step2_df['cloudy'] = step2_df['cloudy'].astype(int)

step2_df['partly_cloudy'] = (step2_df['clear'] == 0) & (step2_df['cloudy'] == 0)
step2_df['partly_cloudy'] = step2_df['partly_cloudy'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,...,not_snowy,very_snowy,lightly_snowy,windy,low_humidity,high_humidity,medium_humidity,clear,cloudy,partly_cloudy
pure_date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,91.375000,20.5,96.625,100,24.0,0.85,0.0,49.7,0,0,...,1,0,0,1,0,1,0,0,1,0
2010-01-02,91.291667,16.6,80.250,100,12.0,0.12,0.0,47.4,1,1,...,1,0,0,1,0,1,0,0,1,0
2010-01-03,91.041667,6.8,84.375,100,2.0,0.02,0.0,47.4,1,1,...,1,0,0,0,0,1,0,0,1,0
2010-01-04,89.541667,8.0,100.000,100,21.0,1.12,0.0,46.0,0,0,...,1,0,0,0,0,1,0,0,1,0
2010-01-05,95.958333,4.7,100.000,100,15.0,0.53,0.0,47.4,0,0,...,1,0,0,0,0,1,0,0,1,0
